In [ ]:
# Install required Python libraries
try:
    import subprocess
    subprocess.check_call(["pip", "install", "torch", "onnx", "numpy", "hummingbird-ml"])
except:
    print("Ensure all dependencies are installed.")

from sklearn.cluster import KMeans
from hummingbird.ml import convert
import numpy as np
import json
import torch

In [ ]:
# Create a dataset of two Gaussians. There will be some overlap
# between the two classes, which adds some uncertainty to the model.
xs = np.concatenate(
    [
        np.random.random(size=(256, 2)) + [1, 0],
        np.random.random(size=(256, 2)) + [-1, 0],
    ],
    axis=0,
)

# Train an SVM on the data and wrap it in PyTorch.
sk_model = KMeans()
sk_model.fit(xs)

# Convert the trained model to ONNX format
onnx_model = convert(sk_model, backend="pytorch").model
onnx_path = "kmeans.onnx"

# Input to the model
spaced = np.linspace(-2, 2, num=25)
grid_xs = torch.tensor([[x, y] for x in spaced for y in spaced], requires_grad=True)
shape = xs.shape[1:]
model_input = grid_xs[0:1]

# Export ONNX model
torch.onnx.export(
    onnx_model, model_input, onnx_path,
    input_names=["input"], output_names=["output"],
    opset_version=13
)
print(f"ONNX model exported to {onnx_path}")


In [ ]:
# Prepare input data for proof generation
data_path = "input.json"

# Generate sample
sample = np.random.random(size=(1, 2)) + [0.5, -0.5]

# Flatten and prepare for JSON serialization
input_data = sample.flatten().tolist()

# Save to JSON file
data = [input_data]  # Wrap in outer array
with open(data_path, "w") as f:
    json.dump(data, f, indent=4)

print(f"Input data saved to {data_path}")


In [ ]:
import subprocess

# Paths for CLI commands
proof_path = "proof.json"

# Command for proof generation
# Make sure to generate the binary first: `cargo build --release`
cmd = [
    "../../target/release/mina-zkml-cli", "proof",
    "-m", onnx_path,
    "-i", data_path,
    "-o", proof_path,
    "--input-visibility", "public",
    "--output-visibility", "public"
]

# Run the CLI command
result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(result.stdout)
if result.returncode == 0:
    print(f"Proof successfully generated at {proof_path}")
else:
    print(f"Error generating proof: {result.stderr}")


In [ ]:
# Extract the "output" field from proof.json
output_path = "output.json"
try:
    with open(proof_path, "r") as proof_file:
        proof_data = json.load(proof_file)
    if "output" in proof_data:
        output_data = proof_data["output"]
        with open(output_path, "w") as output_file:
            json.dump(output_data, output_file, indent=4)
        print(f"Output data successfully saved to {output_path}")
    else:
        print("No 'output' field found in proof.json")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# Create a public output file from the proof
try:
    # Load proof.json
    with open(proof_path, "r") as proof_file:
        proof_data = json.load(proof_file)

    # Extract the "output" field
    if "output" in proof_data:
        output_data = proof_data["output"]
        
        # Save the output data to output.json
        with open(output_path, "w") as output_file:
            json.dump(output_data, output_file, indent=4)
        
        print(f"Output data successfully saved to {output_path}")
    else:
        print("No 'output' field found in proof.json")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Command for proof verification
cmd = [
    "../../target/release/mina-zkml-cli", "verify",
    "-m", onnx_path,
    "-i", data_path,
    "-p", proof_path,
    "-o", output_path,
    "--input-visibility", "public",
    "--output-visibility", "public"
]

# Run the CLI command
result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(result.stdout)
if result.returncode == 0:
    print(f"Proof successfully verified at {proof_path}")
else:
    print(f"Error verifying proof: {result.stderr}")
